In [1]:
import os
import yaml
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from composio_langchain import ComposioToolSet, App, Action
from langchain.memory import ConversationBufferMemory
from langchain.tools import StructuredTool
from langchain_core.tools import ToolException
from typing import List, Optional, Dict, Any
from pydantic import ValidationError
import warnings

warnings.filterwarnings("ignore")

/Users/morossini/anaconda3/envs/composio/lib/python3.12/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/Users/morossini/anaconda3/envs/composio/lib/python3.12/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
class MyAgent:
    def __init__(self, tools, custom_prompt: str = None):
        self.llm = ChatOpenAI(
            model="gpt-4o",
            api_key=os.getenv("OPENAI_API_KEY"),
        )
        self.tools = tools
        self.agent_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.agent = create_tool_calling_agent(
            llm=self.llm,
            tools=self.enhance_agent_tools(tools),
            prompt=ChatPromptTemplate.from_messages([
                ("system", custom_prompt if custom_prompt else "You are an experienced assistant. You are able to perform any task asked by the user through all the tools you have access to. Your goal is to complete the user's task by using the tools you have access to. This is a core instruction: ALWAYS ALWAYS ALWAYS USE A STRING AS ID"),
                ("placeholder", "{chat_history}"),
                ("human", "{input}"),
                ("placeholder", "{agent_scratchpad}"),
            ]),
        )
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, memory=self.agent_memory, verbose=True)

    def myinvoke(self, input: str):
        result = self.executor.invoke({"input": input})
        print(result)
        print("\n==================================================")
        print("--> Agent Response:")
        print(result['output'])
        print("\n==================================================")

    def try_except_tool(self, func):
        try:
            return func
        except ValidationError as e:
            raise ValidationError(f"Validation Error: {e}")
        except Exception as e:
            raise ToolException(f"Error: {e}")

    def enhance_agent_tools(self, agent_tools: List):
        enhanced_agent_tools = [
            StructuredTool(
                name=tool.name,
                description=tool.description,
                args_schema=tool.args_schema,
                func=self.try_except_tool(tool.func),
                handle_tool_error=True,
                handle_validation_error=True
            ) for tool in agent_tools
        ]
        return enhanced_agent_tools

In [3]:
team_id = 12927880
list_id = 901304862476
task_name = "-_-_NOVO TESTE COMPOSIO CLICKUP_-_-"
# task_id = "86a4cmtu8"
# task_id_2 = "86a4cpzc3"
task_id = "blablablalba"
task_id_2 = "fake_id"
folder_id = "54804921"
updated_task_name = "NEW UPDATED TASK NAME"

In [4]:
selected_clickup_actions_with_args = [
    {
        "name": "CLICKUP_AUTHORIZATION_VIEW_ACCOUNT_DETAILS",
        "args": {}
    },
    {
        "name": "CLICKUP_AUTHORIZATION_GET_WORK_SPACE_LIST",
        "args": {}
    },
    {
        "name": "CLICKUP_CREATE_TASK",
        "args": {
            "list_id": list_id,
            "name": task_name,
        }
    },    
    {
        "name": "CLICKUP_GET_TASK",
        "args": {
            "task_id": task_id,
        }
    },
    {
        "name": "CLICKUP_UPDATE_TASK",
        "args": {
            "task_id": task_id,
            "name": updated_task_name,
            "assignees__add": [],
            "assignees__rem": [],
        }
    },
    {
        "name": "CLICKUP_DELETE_TASK",
        "args": {
            "task_id": task_id_2,
        }
    },
    {
        "name": "CLICKUP_GET_TASKS",
        "args": {
            "list_id": list_id,
        }
    },
    {
        "name": "CLICKUP_GET_LIST",
        "args": {
            "list_id": list_id,
        }
    },
    {
        "name": "CLICKUP_GET_LISTS",
        "args": {
            "folder_id": folder_id,
        }
    },
    {
        "name": "CLICKUP_ADD_TASK_TO_LIST",
        "args": {
            "list_id": list_id,
            "task_id": task_id,
        }
    },
    {
        "name": "CLICKUP_REMOVE_TASK_FROM_LIST",
        "args": {
            "list_id": list_id,
            "task_id": task_id,
        }
    },
    {
        "name": "CLICKUP_SPACES_GET_SPACE_DETAILS",
        "args": {
            "team_id": team_id,
        }
    },
    {
        "name": "CLICKUP_SPACES_GET_SPACE_DETAILS",
        "args": {
            "team_id": team_id,
        }
    },
]

In [5]:
from pprint import pprint

composio_toolset = ComposioToolSet()
short_results = []
full_results = []

for action in selected_clickup_actions_with_args:
    try:
        clickup_action = composio_toolset.get_actions(actions=[getattr(Action, action["name"])])
        result = clickup_action[0].invoke(action["args"])
        short_results.append({"action": action["name"], "executed": result['execution_details']})
        full_results.append({"action": action["name"], "results": result})
    except Exception as e:
        short_results.append({"action": action["name"], "executed": result['execution_details']})
        full_results.append({"action": action["name"], "executed": result['execution_details'], "error": str(e)})
    print(short_results[-1])
# pprint(short_results)

{'action': 'CLICKUP_AUTHORIZATION_VIEW_ACCOUNT_DETAILS', 'executed': {'executed': True}}
{'action': 'CLICKUP_AUTHORIZATION_GET_WORK_SPACE_LIST', 'executed': {'executed': True}}
{'action': 'CLICKUP_CREATE_TASK', 'executed': {'executed': True}}
{'action': 'CLICKUP_GET_TASK', 'executed': {'executed': False}}
{'action': 'CLICKUP_UPDATE_TASK', 'executed': {'executed': False}}
{'action': 'CLICKUP_DELETE_TASK', 'executed': {'executed': False}}
{'action': 'CLICKUP_GET_TASKS', 'executed': {'executed': True}}
{'action': 'CLICKUP_GET_LIST', 'executed': {'executed': True}}
{'action': 'CLICKUP_GET_LISTS', 'executed': {'executed': False}}
{'action': 'CLICKUP_ADD_TASK_TO_LIST', 'executed': {'executed': False}}
{'action': 'CLICKUP_REMOVE_TASK_FROM_LIST', 'executed': {'executed': False}}
{'action': 'CLICKUP_SPACES_GET_SPACE_DETAILS', 'executed': {'executed': True}}
{'action': 'CLICKUP_SPACES_GET_SPACE_DETAILS', 'executed': {'executed': True}}


In [6]:
for result in full_results:
    
    pprint(result)
    print("\n=========================================================================================================================\n")

{'action': 'CLICKUP_AUTHORIZATION_VIEW_ACCOUNT_DETAILS',
 'results': {'execution_details': {'executed': True},
             'response_data': {'user': {'color': '#81b1ff',
                                        'email': 'joaog.morossini@gmail.com',
                                        'global_font_support': True,
                                        'id': 81918955,
                                        'initials': 'JM',
                                        'profilePicture': 'https://attachments.clickup.com/profilePictures/81918955_kcK.jpg',
                                        'timezone': 'America/Sao_Paulo',
                                        'username': 'João Guilherme Silva '
                                                    'Morossini',
                                        'week_start_day': 1}}}}


{'action': 'CLICKUP_AUTHORIZATION_GET_WORK_SPACE_LIST',
 'results': {'execution_details': {'executed': True},
             'response_data': {'teams': [{'avatar': 

In [9]:
full_results[0].keys()

dict_keys(['action', 'results'])

In [13]:
type(full_results)

list

In [14]:
full_results[0]

{'action': 'CLICKUP_AUTHORIZATION_VIEW_ACCOUNT_DETAILS',
 'results': {'execution_details': {'executed': True},
  'response_data': {'user': {'id': 81918955,
    'username': 'João Guilherme Silva Morossini',
    'email': 'joaog.morossini@gmail.com',
    'color': '#81b1ff',
    'profilePicture': 'https://attachments.clickup.com/profilePictures/81918955_kcK.jpg',
    'initials': 'JM',
    'week_start_day': 1,
    'global_font_support': True,
    'timezone': 'America/Sao_Paulo'}}}}

In [18]:
full_results[0]['results']['execution_details']['executed']

True

In [ ]:
composio_toolset = ComposioToolSet(api_key=os.getenv('COMPOSIO_API_KEY'))
clickup_action = composio_toolset.get_actions(actions=[Action.CLICKUP_AUTHORIZATION_VIEW_ACCOUNT_DETAILS])
result = clickup_action[0].invoke({})

In [ ]:
list(result['execution_details'].values())[0]

In [ ]:
dir(result['execution_details'].values())

In [ ]:
result['execution_details'].keys()